In [1]:
from bs4 import BeautifulSoup
import requests

compound = "CdSe"
uri  = "https://nano.nature.com/search?workflow=article&term=freeText%3A%22"+compound+"%22&sort-by=relevance"
uri2 = "https://nano.nature.com/search?term=freeText%3A%22ZnTe%22&sort-by=relevance&page-number=2&workflow=article"

r1 = requests.get(uri)
coverpage = r1.content

In [3]:
soup1 = BeautifulSoup(coverpage, 'lxml')

In [44]:
td = soup1.find_all('li', class_="FacetValues_value") # Find all the td elements on the page
mls = []
for el in td:  
    mls.append(el)

In [48]:
type(mls[0])

bs4.element.Tag

In [ ]:
b  